In [3]:
import pandas as pd
import pymysql

# Establish connection to MySQL
myconnection = pymysql.connect(host='127.0.0.1', user='root', passwd='Admin@Yunus111', database='project_1')

# Read the CSV file
csv_file_path = r'C:\Users\HP\redbus_all_red.csv'
df = pd.read_csv(csv_file_path)

# Print the columns to debug
print("Columns in DataFrame:", df.columns)

# Ensure correct column names and drop unnecessary columns
df = df.rename(columns=lambda x: x.strip())  # Remove leading/trailing whitespaces

# Rename columns to match expected names
df.rename(columns={
    'Bus_name': 'Bus_Name',
    'Bus Type': 'Bus_Type',
    'Departure Time': 'Departure_Time',
    'Arrival Time': 'Arrival_Time',
    'Seat_Availability': 'Seat_Availability',
    'Bus Link': 'Bus_Link',
    'Bus Route': 'Bus_Route'
}, inplace=True)

# Format the DataFrame columns
try:
    if 'Departure_Time' in df.columns:
        df['Departure_Time'] = pd.to_datetime(df['Departure_Time'], format='%H:%M', errors='coerce').dt.time
    if 'Arrival_Time' in df.columns:
        df['Arrival_Time'] = pd.to_datetime(df['Arrival_Time'], format='%H:%M', errors='coerce').dt.time
except Exception as e:
    print(f"Error parsing date columns: {e}")

# Clean 'Seat_Availability' column and handle NaN values
if 'Seat_Availability' in df.columns:
    df['Seat_Availability'] = df['Seat_Availability'].astype(str).str.extract(r'(\d+)').fillna(0).astype(int)
else:
    print("Warning: 'Seat_Availability' column not found")

# Ensure 'Rating' and 'Fare' columns are properly formatted
if 'Rating' in df.columns:
    df['Rating'] = pd.to_numeric(df['Rating'], errors='coerce').fillna(0)  # Convert to float and fill NaNs with 0
if 'Fare' in df.columns:
    df['Fare'] = pd.to_numeric(df['Fare'], errors='coerce').fillna(0)  # Convert to float and fill NaNs with 0

# Ensure 'Bus_Name' column is present and treated as string
if 'Bus_Name' in df.columns:
    df['Bus_Name'] = df['Bus_Name'].astype(str).fillna('Unknown')
else:
    print("Warning: 'Bus_Name' column not found")

# Map pandas dtypes to MySQL dtypes
dtype_mapping = {
    'object': 'TEXT',
    'int64': 'INT',
    'int32': 'INT',
    'float64': 'FLOAT',
    'datetime64[ns]': 'DATETIME',
    'datetime.time': 'TIME'
}

# Replace spaces with underscores in column names
df.columns = df.columns.str.replace(' ', '_')

# Generate the formatted string for creating the table with escaped column names and types
column_definitions = [f"`{col}` {dtype_mapping.get(str(df[col].dtype), 'TEXT')}" for col in df.columns]
table_definition = ", ".join(column_definitions)

table_name = "redbus"

# Create cursor object
cursor = myconnection.cursor()

# Drop table if it exists (optional)
cursor.execute(f"DROP TABLE IF EXISTS {table_name}")

# Create table
create_table_query = f"CREATE TABLE {table_name} ({table_definition})"
print("Create Table Query:", create_table_query)  # Print the query to debug
cursor.execute(create_table_query)

# Insert data into the table
insert_query = f"INSERT INTO {table_name} ({', '.join(df.columns)}) VALUES ({', '.join(['%s'] * len(df.columns))})"

# Execute the insert queries
for row in df.itertuples(index=False, name=None):
    cursor.execute(insert_query, row)

# Commit the changes
myconnection.commit()

# Close the cursor and connection
cursor.close()
myconnection.close()


Columns in DataFrame: Index(['Bus_name', 'Bus Type', 'Departure Time', 'Arrival Time', 'Duration',
       'Fare', 'Rating', 'Seat_Availability', 'Bus Link', 'Bus Route'],
      dtype='object')
Create Table Query: CREATE TABLE redbus (`Bus_Name` TEXT, `Bus_Type` TEXT, `Departure_Time` TEXT, `Arrival_Time` TEXT, `Duration` TEXT, `Fare` FLOAT, `Rating` FLOAT, `Seat_Availability` INT, `Bus_Link` TEXT, `Bus_Route` TEXT)


In [4]:
df

,Bus_Name,Bus_Type,Departure_Time,Arrival_Time,Duration,Fare,Rating,Seat_Availability,Bus_Link,Bus_Route
0,Kadamba Transport Corporation Limited (KTCL) -...,Volvo AC Seater 2+2,19:15:00,06:30:00,11h 15m,800.0,4.5,23,https://www.redbus.in/bus-tickets/pune-to-goa,Pune to Goa
1,Atmaram Gobus,VE A/C Sleeper (2+1),22:30:00,08:45:00,10h 15m,606.0,4.7,10,https://www.redbus.in/bus-tickets/pune-to-goa,Pune to Goa
2,Atmaram Gobus,VE A/C Sleeper (2+1),21:00:00,07:30:00,10h 30m,555.0,4.6,17,https://www.redbus.in/bus-tickets/pune-to-goa,Pune to Goa
3,AdIntrCity SmartBus,AC Sleeper (2+1),21:00:00,07:40:00,10h 40m,500.0,4.1,14,https://www.redbus.in/bus-tickets/pune-to-goa,Pune to Goa
4,Ashray Amrutyog,A/C Sleeper (2+1),22:00:00,09:30:00,11h 30m,649.0,4.5,19,https://www.redbus.in/bus-tickets/pune-to-goa,Pune to Goa
...,...,...,...,...,...,...,...,...,...,...
1566,Express Line,Volvo 9600 Multi Axle Semi-Sleeper (2+2),15:30:00,19:40:00,04h 10m,524.0,4.5,0,https://www.redbus.in/bus-tickets/kolkata-to-a...,Kolkata to Asansol (West Bengal)
1567,Express Line (Karunamoyee),Volvo 9600 Multi Axle Semi-Sleeper (2+2),15:45:00,20:45:00,05h 00m,524.0,4.8,0,https://www.redbus.in/bus-tickets/kolkata-to-a...,Kolkata to Asansol (West Bengal)
1568,Greenline,Volvo 9600 Multi Axle Semi-Sleeper (2+2),16:00:00,20:00:00,04h 00m,524.0,4.4,0,https://www.redbus.in/bus-tickets/kolkata-to-a...,Kolkata to Asansol (West Bengal)
1569,SHYAMOLI Pvt. Ltd.(Karunamoyee),Volvo A/C Seater (2+2),16:00:00,21:00:00,05h 00m,524.0,4.1,0,https://www.redbus.in/bus-tickets/kolkata-to-a...,Kolkata to Asansol (West Bengal)
